In [1]:
import os
import sys
os.chdir('../')
sys.path.append(os.path.join(os.getcwd(), "src"))

In [2]:
from dataclasses import dataclass
from pathlib import Path
from WattPredictor.utils.helpers import *
from WattPredictor.utils.exception import *
from WattPredictor.constants import *
from WattPredictor.utils.logging import logger

In [3]:
import pandas as pd
from datetime import datetime, timedelta
from WattPredictor.utils.exception import CustomException
from WattPredictor.utils.feature import feature_store_instance
from WattPredictor.config.inference_config import InferenceConfigurationManager
from WattPredictor.utils.helpers import create_directories
from WattPredictor.entity.config_entity import MonitoringConfig
from WattPredictor.utils.logging import logger

class Monitoring:
    def __init__(self, config: MonitoringConfig):
        self.config = config
        self.feature_store = feature_store_instance()

    def predictions_and_actuals(self):
        predictions_fg = self.feature_store.feature_store.get_feature_group(
            name=self.config.predictions_fg_name,
            version=self.config.predictions_fg_version
        )
        actuals_fg = self.feature_store.feature_store.get_feature_group(
            name=self.config.actuals_fg_name,
            version=self.config.actuals_fg_version
        )
        predictions_df = predictions_fg.read()
        actuals_df = actuals_fg.read()

        predictions_df['date'] = pd.to_datetime(predictions_df['date']).dt.tz_convert('UTC')
        actuals_df['date'] = pd.to_datetime(actuals_df['date']).dt.tz_convert('UTC')

        from_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d 00:00:00")
        to_date = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d 23:59:59")

        combined_df = pd.merge(
            predictions_df,
            actuals_df[['sub_region_code', 'date', 'demand']],
            on=['sub_region_code', 'date']
        )
        mask = (combined_df['date'] >= from_date) & (combined_df['date'] <= to_date)
        monitoring_df = combined_df.loc[mask].sort_values(by=['date', 'sub_region_code'])

        create_directories([self.config.monitoring_df.parent])
        monitoring_df.to_csv(self.config.monitoring_df, index=False)
        logger.info(f"Monitoring data saved to {self.config.monitoring_df}")
        return monitoring_df

In [4]:
config = InferenceConfigurationManager()
monitor_config = config.get_data_monitoring_config()
monitor = Monitoring(config=monitor_config)
monitor.predictions_and_actuals()

[2025-07-20 20:00:49,010: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-20 20:00:49,016: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-07-20 20:00:49,019: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]
[2025-07-20 20:00:49,021: INFO: helpers: created directory at: artifacts]
[2025-07-20 20:00:49,029: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-20 20:00:49,032: INFO: external: Initializing external client]
[2025-07-20 20:00:49,033: INFO: external: Base URL: https://c.app.hopsworks.ai:443]
[2025-07-20 20:00:52,152: INFO: python: Python Engine initialized.]

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1240214
[2025-07-20 20:00:54,892: INFO: feature_store: Connected to Hopsworks Feature Store: WattPredictor]
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.84s) 
Finished: Reading data from Hopsworks, using Hop

,sub_region_code,predicted_demand,date,demand
3,0,1677.0,2025-07-19 04:00:00+00:00,1735.0
10,1,965.0,2025-07-19 04:00:00+00:00,1003.0
4,2,1624.0,2025-07-19 04:00:00+00:00,1659.0
5,3,558.0,2025-07-19 04:00:00+00:00,552.0
2,4,816.0,2025-07-19 04:00:00+00:00,858.0
1,5,1380.0,2025-07-19 04:00:00+00:00,1389.0
6,6,1181.0,2025-07-19 04:00:00+00:00,1217.0
7,7,305.0,2025-07-19 04:00:00+00:00,320.0
9,8,725.0,2025-07-19 04:00:00+00:00,778.0
0,9,7298.0,2025-07-19 04:00:00+00:00,6783.0
